## Transfer POI from on knee to another

This jupiter notebook shows you_
- how to make a single Leg point atlas file.
- how to registrate these point to a other segmentation.
- how to add new Points to the atlas

We expect that you have the same numbering as Network 10 from https://github.com/robert-graf/TotalVibeSegmentator

- sacrum: 11,
- hip: 12,
- femur: 13,
- patella: 14,
- tibia: 15,
- fibula: 16,
- talus: 17,
- calcaneus: 18,
- tarsals: 19,
- metatarsals: 20,
- phalanges: 21,
- trachea: 22,

### Installation
torch : https://pytorch.org/get-started/locally/

```bash
pip install TPTBox

pip install hf-deepali
```


In [ ]:
%load_ext autoreload
%autoreload 2
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "3"


In [ ]:
from TPTBox.segmentation.TotalVibeSeg import run_totalvibeseg

path_to_ct = "..."
out_seg = "seg.nii.gz"
ddevice = "cpu" #"cuda" / "mps"
run_totalvibeseg(path_to_ct,out_seg=out_seg,override=False,gpu=1,ddevice=ddevice,dataset_id=10,keep_size=True)

# Making a new atlas

You need a poi file or a text file in this format:

```bash
TGT (139.4188232421875, -92.69547271728516, 1138.0546875)
FHC (93.09368133544922, -108.62625885009766, 1136.891845703125)
FNC (107.05316162109375, -107.46392059326172, 1121.36865234375)
FAAP (110.07427978515625, -110.30272674560547, 1027.859130859375)
FLCD (89.51376342773438, -108.07373046875, 732.1410522460938)
FMCD (41.474029541015625, -103.25114440917969, 731.2152709960938)
FLCP (92.19694519042969, -84.93902587890625, 751.868896484375)
FMCP (42.786216735839844, -74.31689453125, 753.4283447265625)
FNP (63.380027770996094, -116.54719543457031, 739.7014770507812)
FADP (71.667724609375, -121.2674560546875, 822.39990234375)
TGPP (60.06828308105469, -132.88507080078125, 758.1168823242188)
TGCP (61.84690475463867, -131.874267578125, 751.1783447265625)
FMCPC (46.220550537109375, -83.17189025878906, 767.088623046875)
FLCPC (89.6451187133789, -92.42149353027344, 766.2191162109375)
TRMP (46.96009826660156, -131.4429931640625, 748.450927734375)
TRLP (79.27147674560547, -142.57913208007812, 754.849365234375)
FLM (73.47111511230469, -97.508544921875, 384.16046142578125)
TMM (19.230003356933594, -129.28802490234375, 393.071533203125)
TAC (41.175079345703125, -114.86720275878906, 392.0289001464844)
TADP (50.04762268066406, -116.96440887451172, 459.7703857421875)
TLCL (99.41020202636719, -108.22544860839844, 727.647705078125)
TMCM (34.0052490234375, -97.65898132324219, 727.727783203125)
TKC (67.44658660888672, -99.75123596191406, 734.62744140625)
TLCA (88.45868682861328, -119.93305969238281, 729.4689331054688)
TLCP (91.51274871826172, -91.85955047607422, 726.2589721679688)
TMCA (42.026023864746094, -118.33468627929688, 732.6619262695312)
TMCP (48.68177032470703, -83.51608276367188, 726.7157592773438)
TTP (70.8026123046875, -137.39186096191406, 697.2947998046875)
TAAP (62.44767761230469, -117.48356628417969, 654.552490234375)
TMIT (60.547447204589844, -99.73310852050781, 736.443359375)
TLIT (73.91001892089844, -99.01293182373047, 735.8458251953125)
PPP (60.099151611328125, -142.22222900390625, 776.0038452148438)
PDP (62.95762634277344, -146.806884765625, 739.1683959960938)
PMP (43.483184814453125, -149.5174560546875, 754.05224609375)
PLP (80.17727661132812, -150.05401611328125, 762.3734130859375)
PRPP (59.09102249145508, -141.71621704101562, 775.1181030273438)
PRDP (57.41969680786133, -137.11688232421875, 748.3690185546875)
PRHP (61.70811462402344, -138.3133544921875, 751.9981079101562)
```


We assume that all atlases will be flipped so they will be always a "left leg"

In [ ]:
#Step 1
from pathlib import Path

from atlas_poi_transfer import parse_coordinates_to_poi, prep_Atlas

from TPTBox import POI, to_nii
from TPTBox.core.vert_constants import Full_Body_Instance, Lower_Body

##########################################
#Settings
text_file_is_left_leg = True
file_text = "010__left.txt"
segmentation_path = "/DATA/NAS/datasets_processed/CT_fullbody/dataset-watrinet/source/Dataset001_all/0001/bone.nii.gz"
out_folder = Path("/DATA/NAS/datasets_processed/CT_fullbody/dataset-watrinet/atlas")
atlas_id = 1
##########################################
# Load segmentation
seg = to_nii(segmentation_path,True)

if not text_file_is_left_leg:
    axis = seg.get_axis("R")
    if axis == 0:
        target = seg.set_array(seg.get_array()[::-1]).copy()
    elif axis == 1:
        target = seg.set_array(seg.get_array()[:, ::-1]).copy()
    elif axis == 2:
        target = seg.set_array(seg.get_array()[:, :, ::-1]).copy()
assert text_file_is_left_leg, "Not implement: Flip NII and POI"
# Prep atlas
atlas_path = out_folder/f"atlas{atlas_id:03}.nii.gz"
atlas_cms_poi_path = out_folder/f"atlas{atlas_id:03}_cms_poi.json" # Center of mass
atlas_poi_path = out_folder/f"atlas{atlas_id:03}_poi.json"
prep_Atlas(target,atlas_path,atlas_cms_poi_path,text_file_is_left_leg)


poi = parse_coordinates_to_poi(file_text, True).to_other(seg) if ".txt" in file_text else POI.load(file_text).resample_from_to(seg)
if not text_file_is_left_leg:
    for k1, k2, (x, y, z) in poi.items():
        axis = poi.get_axis("R")
        if axis == 0:
            poi[k1, k2] = (poi.shape[0] - 1 - x, y, z)
        elif axis == 1:
            poi[k1, k2] = (x, poi.shape[1] - 1 - y, z)
        elif axis == 2:
            poi[k1, k2] = (x, y, poi.shape[2] - 1 - z)
        else:
            raise ValueError(axis)
poi.level_one_info=Full_Body_Instance
poi.level_two_info=Lower_Body
poi.to_global().save(atlas_poi_path)


Check if the other leg was removed from the atlas.

In [ ]:
from IPython.display import Image

from TPTBox.spine.snapshot2D import Snapshot_Frame, Visualization_Type, create_snapshot

out_file = "out.jpg"
create_snapshot(out_file,[Snapshot_Frame(atlas_path,atlas_path,atlas_poi_path,visualization_type = Visualization_Type.Mean_Intensity,coronal=True)])
Image(filename=out_file)

## Transferring POI to new Segmentation

In [ ]:
#Step 1
from pathlib import Path

from atlas_poi_transfer import Register_Point_Atlas

from TPTBox import POI, to_nii

##########################################
#Settings
target_seg_path = "/DATA/NAS/datasets_processed/CT_fullbody/dataset-watrinet/source/Dataset001_all/0003/bone.nii.gz"#TODO Path to target seg
s = str(target_seg_path).split('.')[0]
split_leg_path = s + "_seg-left-right-split_msk.nii.gz"
out_new_pois = s + "_desc-leg_poi.json"
out_new_pois_nii = s + "_desc-leg_poi.nii.gz"
atlas_id = 1
ddevice = "cuda"
gpu = 0
##########################################
# Atlas
atlas_p = out_folder/f"atlas{atlas_id:03}.nii.gz"
atlas_centroids = out_folder/f"atlas{atlas_id:03}_cms_poi.json" # Center of mass
atlas_poi_path = out_folder/f"atlas{atlas_id:03}_poi.json"
# Load segmentation
target = to_nii(target_seg_path,True)
atlas = to_nii(atlas_p,True)

#Creating this object will start the registration
registration_obj = Register_Point_Atlas(target,atlas,split_leg_path=split_leg_path,atlas_centroids=atlas_centroids,gpu=gpu,ddevice=ddevice, verbose=0)

In [ ]:
out_poi = registration_obj.make_poi_from_poi(POI.load(atlas_poi_path),out_new_pois)
nii = out_poi.make_point_cloud_nii()[1] + to_nii(split_leg_path,True)*100
nii.save(out_new_pois_nii)

## save
You can save and load the Registration instead of recomputing it. 

In [ ]:
registration_obj.save("registration_obj.pkl",compress=True)

registration_obj_new = Register_Point_Atlas.load("registration_obj.pkl")